In [1]:
import sympy as sp
from sympy import pi, Eq, solve

In [2]:
Xs, Ys, Xz, Yz, fg, K, s, z, Ts = sp.symbols('X(s) Y(s) X(z) Y(z) f_g K s z T_s')

## Dedução da função de transferência do SOGI-PLL no domínio $s$

![Diagrama-SOGI-PLL](diagrama_sogi_pll.png)

In [3]:
E1 = Xs-Ys
E2 = K*E1-(2*pi*fg/s)*Ys
eq_y = sp.Eq(Ys, 2*pi*fg*E2/s); eq_y

Eq(Y(s), 2*pi*f_g*(K*(X(s) - Y(s)) - 2*pi*Y(s)*f_g/s)/s)

In [4]:
eq_tf_s = Eq(Ys/Xs, solve(eq_y, Ys)[0]/Xs)
print("Função de transferência em s do SOGI-PLL:"); display(eq_tf_s)

Função de transferência em s do SOGI-PLL:


Eq(Y(s)/X(s), 2*pi*K*f_g*s/(2*pi*K*f_g*s + 4*pi**2*f_g**2 + s**2))

## Conversão da função de transferência do SOGI-PLL para o domínio $z$

In [5]:
def z_transform(eq: Eq):
    rhs = eq.rhs.subs({s: (2/Ts)*(z-1)/(z+1)}).simplify()
    lhs = Yz/Xz
    return Eq(lhs, rhs)

In [6]:
eq_tf_z = z_transform(eq_tf_s)
print("Função de transferência em z do SOGI-PLL:"); display(eq_tf_z)

Função de transferência em z do SOGI-PLL:


Eq(Y(z)/X(z), pi*K*T_s*f_g*(z - 1)*(z + 1)/(pi*K*T_s*f_g*(z - 1)*(z + 1) + pi**2*T_s**2*f_g**2*(z + 1)**2 + (z - 1)**2))

In [7]:
rhs_numerator = eq_tf_z.rhs.as_numer_denom()[0]
rhs_denominator = eq_tf_z.rhs.as_numer_denom()[1]
lhs_numerator = eq_tf_z.lhs.as_numer_denom()[0]
lhs_denominator = eq_tf_z.lhs.as_numer_denom()[1]

eq_expanded = Eq(lhs_numerator*rhs_denominator, rhs_numerator*lhs_denominator).expand().simplify()
eq_expanded

Eq(pi*K*T_s*X(z)*f_g*(z**2 - 1), Y(z)*(pi*K*T_s*f_g*z**2 - pi*K*T_s*f_g + pi**2*T_s**2*f_g**2*z**2 + 2*pi**2*T_s**2*f_g**2*z + pi**2*T_s**2*f_g**2 + z**2 - 2*z + 1))

In [16]:
eq_offset = Eq(eq_expanded.lhs/z**2, eq_expanded.rhs/z**2).expand().simplify(); eq_offset

Eq(pi*K*T_s*X(z)*f_g*(z**2 - 1)/z**2, Y(z)*(-pi*K*T_s*f_g + pi**2*T_s**2*f_g**2 + z**2*(pi*K*T_s*f_g + pi**2*T_s**2*f_g**2 + 1) + 2*z*(pi**2*T_s**2*f_g**2 - 1) + 1)/z**2)

In [9]:
eq_subs = eq_offset.subs({
    Ts: 1e-4,
    K: 0.2,
    fg: 60
}).evalf(6)
eq_subs

Eq(0.00376991*X(z) - 0.00376991*X(z)/z**2, 1.00413*Y(z) - 1.99929*Y(z)/z + 0.996585*Y(z)/z**2)